In [1]:
import os
import json
import pandas as pd
import numpy as np
import torch
import evaluate
from datasets import Dataset, Audio, load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments, TrainerCallback
from dataclasses import dataclass
from typing import Dict, List, Optional, Union, Any
import re

# 메모리 단편화 방지 설정
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 데이터 경로 설정
train_audio_dir = "/home/ace3_yongjae/speechRecog/train/training"
train_json_dir = "/home/ace3_yongjae/speechRecog/train/labeling"
valid_audio_dir = "/home/ace3_yongjae/speechRecog/valid/validation" 
valid_json_dir = "/home/ace3_yongjae/speechRecog/valid/labeling"

# JSON 파일 로드 함수
def load_json_files(directory):
    json_data = []
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as f:
                data = json.load(f)
                json_data.append(data)
    return json_data

# 학습 및 검증 데이터 JSON 로드
train_json = load_json_files(train_json_dir)
valid_json = load_json_files(valid_json_dir)

# 데이터프레임 생성
def create_dataframe(json_data, audio_dir):
    data = []
    for item in json_data:
        # JSON에서 필요한 정보 추출
        file_name = item.get('fileName')
        answer_text = item.get('transcription', {}).get('AnswerLabelText', '')
        
        # 파일 경로 생성
        audio_path = os.path.join(audio_dir, file_name)
        
        # 데이터가 유효한지 확인 (파일 존재 및 텍스트가 있는지)
        if os.path.exists(audio_path) and answer_text:
            data.append({
                'file_path': audio_path,
                'text': answer_text
            })
    
    return pd.DataFrame(data)

# 학습 및 검증 데이터프레임 생성
train_df = create_dataframe(train_json, train_audio_dir)
valid_df = create_dataframe(valid_json, valid_audio_dir)

# 라벨 전처리 함수 (공백 포함, 한글+공백만 남김)
def prepare_korean_text(text):
    # 한글, 공백, 기본 구두점만 남기고 정규화
    text = re.sub(r'[^\uAC00-\uD7A3\s.,!?]', '', text)  # Keep punctuation
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train_df['normalized_text'] = train_df['text'].apply(prepare_korean_text)
valid_df['normalized_text'] = valid_df['text'].apply(prepare_korean_text)

print(f"학습 데이터 크기: {len(train_df)}")
print(f"검증 데이터 크기: {len(valid_df)}")
print(train_df[['text', 'normalized_text']].head())

/home/ace3_yongjae/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


학습 데이터 크기: 1777
검증 데이터 크기: 222
                                                text  \
0  가장 마지막으로 받은 편지는 두 개 받은 편지는 기억이 안나요 기억이 안 나니까 편...   
1  사실 한번 가보고 싶은 나라는 한국이에요 왜냐하면 저는 지금 한국어를 배우고 있고 ...   
2  사실은 제가 내일은 딱히 계획은 없어요 아침에 제 제 이 백신 맞으러 갈 거고 어/...   
3    네 산책하는 것을 좋아합니다 산책할 때는 공기가 참 좋고 나무들을 보는 게 즐겁습니다   
4  드라마와 영화 중에 제가 드라마 더 좋아합니다 왜냐면 드라마는 쪼끔 더 길겠지만 매...   

                                     normalized_text  
0  가장 마지막으로 받은 편지는 두 개 받은 편지는 기억이 안나요 기억이 안 나니까 편...  
1  사실 한번 가보고 싶은 나라는 한국이에요 왜냐하면 저는 지금 한국어를 배우고 있고 ...  
2  사실은 제가 내일은 딱히 계획은 없어요 아침에 제 제 이 백신 맞으러 갈 거고 어 ...  
3    네 산책하는 것을 좋아합니다 산책할 때는 공기가 참 좋고 나무들을 보는 게 즐겁습니다  
4  드라마와 영화 중에 제가 드라마 더 좋아합니다 왜냐면 드라마는 쪼끔 더 길겠지만 매...  


In [2]:
# 사전학습 모델의 토크나이저와 프로세서 직접 불러오기
MODEL_ID = "kresnik/wav2vec2-large-xlsr-korean"
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)

# 모델 불러오기 (사전학습된 모델의 토크나이저를 그대로 사용하므로 ignore_mismatched_sizes 필요 없음)
model = Wav2Vec2ForCTC.from_pretrained(
    MODEL_ID,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id
)

# 처음에는 feature encoder를 고정
model.freeze_feature_encoder()

In [3]:
# 데이터셋 준비
def prepare_dataset(df):
    dataset = Dataset.from_pandas(df)
    dataset = dataset.cast_column("file_path", Audio(sampling_rate=16000))
    return dataset

train_dataset = prepare_dataset(train_df)
valid_dataset = prepare_dataset(valid_df)

# 샘플 확인
print(train_dataset[0])

# 데이터 전처리 함수
def prepare_dataset_for_model(batch):
    # 오디오 로드 및 처리
    audio = batch["file_path"]
    
    # 샘플링 레이트 검증
    if audio["sampling_rate"] != 16000:
        print(f"Warning: Expected sampling rate 16000, got {audio['sampling_rate']}")
    
    # 오디오 정규화
    array = audio["array"]
    if np.max(np.abs(array)) > 0:
        array = array / np.max(np.abs(array))
    
    batch["input_values"] = processor(
        array, 
        sampling_rate=16000
    ).input_values[0]
    
    # 텍스트 토큰화 - 기존 모델의 토크나이저 사용
    with processor.as_target_processor():
        batch["labels"] = processor(batch["normalized_text"]).input_ids
    
    return batch

# 데이터셋에 전처리 함수 적용
train_dataset = train_dataset.map(prepare_dataset_for_model, remove_columns=train_dataset.column_names)
valid_dataset = valid_dataset.map(prepare_dataset_for_model, remove_columns=valid_dataset.column_names)

print(train_dataset.features)

{'file_path': {'path': '/home/ace3_yongjae/speechRecog/train/training/EN10QC268_EN0171_20211101.wav', 'array': array([0., 0., 0., ..., 0., 0., 0.], shape=(216956,)), 'sampling_rate': 16000}, 'text': '가장 마지막으로 받은 편지는 두 개 받은 편지는 기억이 안나요 기억이 안 나니까 편지 내용도 기억이 안 나요', 'normalized_text': '가장 마지막으로 받은 편지는 두 개 받은 편지는 기억이 안나요 기억이 안 나니까 편지 내용도 기억이 안 나요'}


Map:   0%|          | 0/1777 [00:00<?, ? examples/s]/home/ace3_yongjae/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 222/222 [00:00<00:00, 379.59 examples/s]

{'input_values': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


In [4]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    CTC를 위한 데이터 정렬기
    """
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 입력 값과 레이블 분리
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # 배치 패딩
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        
        # 레이블 패딩
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # 배치에 레이블 추가
        batch["labels"] = labels_batch["input_ids"]

        return batch

# 데이터 정렬기 생성
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [5]:
# WER과 CER 평가 메트릭 로드
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    
    # -100을 패드 토큰 ID로 변경
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    
    # 예측 및 정답 디코딩
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    
    # WER 및 CER 계산
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer, "cer": cer}

# Feature Encoder를 5에폭 이후에 언프리즈하기 위한 콜백
class UnfreezeFeatureEncoderCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, **kwargs):
        if state.epoch == 5:
            model = kwargs.get('model', None)
            if model is not None:
                model.wav2vec2.feature_extractor._freeze_parameters = False
                for param in model.wav2vec2.feature_extractor.parameters():
                    param.requires_grad = True
                print("\n특징 추출기(Feature Encoder)가 언프리즈 되었습니다!")
    
    # 에폭 끝에 GPU 캐시 정리
    def on_epoch_end(self, args, state, control, **kwargs):
        torch.cuda.empty_cache()
        print(f"\n에폭 {state.epoch} 완료, GPU 캐시 정리됨")

In [6]:
# 학습 인자 및 Trainer
training_args = TrainingArguments(
    output_dir="./wav2vec2-korean-asr",
    group_by_length=True,
    per_device_train_batch_size=2,  # 배치 사이즈 증가
    per_device_eval_batch_size=2,   # 배치 사이즈 증가
    gradient_accumulation_steps=8,  # 유효 배치 크기 유지
    eval_strategy="steps",
    num_train_epochs=30,
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    learning_rate=3e-4,  # 학습률 약간 상향
    weight_decay=0.005,
    warmup_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=processor,
    callbacks=[UnfreezeFeatureEncoderCallback()]
)

/tmp/ipykernel_128340/4275728601.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [7]:
# 모델 학습 및 저장
trainer.train()

model.save_pretrained("./final-model")
processor.save_pretrained("./final-model")

# 모델 평가
eval_results = trainer.evaluate(valid_dataset)

print("\n========== 모델 평가 결과 ==========")
print(f"평가 손실 (eval_loss): {eval_results.get('eval_loss'):.4f}")
print(f"평가 WER (eval_wer): {eval_results.get('eval_wer'):.4f}")
print(f"평가 CER (eval_cer): {eval_results.get('eval_cer'):.4f}")
print(f"평가 런타임 (초): {eval_results.get('eval_runtime'):.2f}")
print(f"초당 평가 샘플 수: {eval_results.get('eval_samples_per_second'):.2f}")
print(f"초당 평가 스텝 수: {eval_results.get('eval_steps_per_second'):.2f}")
print(f"평가 에포크: {eval_results.get('epoch'):.2f}")
print("====================================")

/home/ace3_yongjae/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ace3_yongjae/.venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer,Cer
500,-0.086200,0.061471,0.406181,0.204557



에폭 0.968609865470852 완료, GPU 캐시 정리됨

에폭 1.9686098654708521 완료, GPU 캐시 정리됨

에폭 2.968609865470852 완료, GPU 캐시 정리됨

에폭 3.968609865470852 완료, GPU 캐시 정리됨

에폭 4.968609865470852 완료, GPU 캐시 정리됨

에폭 5.968609865470852 완료, GPU 캐시 정리됨

에폭 6.968609865470852 완료, GPU 캐시 정리됨

에폭 7.968609865470852 완료, GPU 캐시 정리됨

에폭 8.968609865470851 완료, GPU 캐시 정리됨

에폭 9.968609865470851 완료, GPU 캐시 정리됨

에폭 10.968609865470851 완료, GPU 캐시 정리됨

에폭 11.968609865470851 완료, GPU 캐시 정리됨

에폭 12.968609865470851 완료, GPU 캐시 정리됨

에폭 13.968609865470851 완료, GPU 캐시 정리됨

에폭 14.968609865470851 완료, GPU 캐시 정리됨

에폭 15.968609865470851 완료, GPU 캐시 정리됨

에폭 16.968609865470853 완료, GPU 캐시 정리됨

에폭 17.968609865470853 완료, GPU 캐시 정리됨


/home/ace3_yongjae/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ace3_yongjae/.venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



에폭 18.968609865470853 완료, GPU 캐시 정리됨

에폭 19.968609865470853 완료, GPU 캐시 정리됨

에폭 20.968609865470853 완료, GPU 캐시 정리됨

에폭 21.968609865470853 완료, GPU 캐시 정리됨

에폭 22.968609865470853 완료, GPU 캐시 정리됨

에폭 23.968609865470853 완료, GPU 캐시 정리됨

에폭 24.968609865470853 완료, GPU 캐시 정리됨

에폭 25.968609865470853 완료, GPU 캐시 정리됨

에폭 26.968609865470853 완료, GPU 캐시 정리됨

에폭 27.968609865470853 완료, GPU 캐시 정리됨

에폭 28.968609865470853 완료, GPU 캐시 정리됨

에폭 29.968609865470853 완료, GPU 캐시 정리됨


/home/ace3_yongjae/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ace3_yongjae/.venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



========== 모델 평가 결과 ==========
평가 손실 (eval_loss): 0.0391
평가 WER (eval_wer): 0.4066
평가 CER (eval_cer): 0.2043
평가 런타임 (초): 22.02
초당 평가 샘플 수: 10.08
초당 평가 스텝 수: 1.27
평가 에포크: 29.97


In [9]:
# 모델과 프로세서를 함께 저장
model.save_pretrained("./save-model")
processor.save_pretrained("./save-model")

# 추가 메타데이터 저장
torch.save({
    'model_state_dict': model.state_dict(),
    'training_args': training_args
}, "./save-model/additional_info.pth")

In [11]:
# 테스트 추론 예제
import torch
import librosa

def transcribe_audio(audio_path, model, processor):
    audio, rate = librosa.load(audio_path, sr=16000)
    input_values = processor(audio, sampling_rate=rate, return_tensors="pt").input_values
    if torch.cuda.is_available():
        input_values = input_values.to("cuda")
        model = model.to("cuda")
    with torch.no_grad():
        logits = model(input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)
    print(f"Raw pred_ids: {pred_ids}")
    transcription = processor.batch_decode(pred_ids)[0]
    return transcription

# 테스트 예제
test_file = "/home/ace3_yongjae/speechRecog/valid/validation/EN10QC227_EN0101_20211108.wav"
if os.path.exists(test_file):
    transcription = transcribe_audio(test_file, model, processor)
    print(f"인식 결과: {transcription}")
else:
    print(f"테스트 파일을 찾을 수 없습니다: {test_file}")


Raw pred_ids: tensor([[1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,  752, 1204, 1204, 1204,
         1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,  167, 1204, 1204, 1204,
         1204, 1204, 1204, 1204,  804, 1204, 1204, 1204, 1204, 1204, 1204,  859,
          859, 1204, 1204, 1204, 1204,  406, 1204, 1204, 1204, 1204, 1204,  859,
          859, 1204, 1204, 1204, 1204, 1204,  459, 1204, 1204, 1204, 1204, 1204,
         1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1083, 1204,
         1204, 1204, 1204, 1204, 1204, 1204,   86, 1204, 1204, 1204, 1204, 1204,
          859,  859, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204,

In [ ]:
#나중에 모델 로드 예시
model = Wav2Vec2ForCTC.from_pretrained("./final-model")
processor = Wav2Vec2Processor.from_pretrained("./final-model")